<h1 style='color:red'> K-Nearest neighbours </h1>

##### Nearest Neighbours to a Movie

In [18]:
import pandas as pd

In [19]:
r_cols = [ "user_id","movie_id","ratings"]

In [20]:
ratings = pd.read_csv("ml-100k/u.data",sep="\t",
                     names=r_cols,usecols=range(3))
ratings.head()

,user_id,movie_id,ratings
0,0,50,5.0
1,0,172,NaN
2,0,133,1.0
3,196,242,3.0
4,186,302,3.0


In [21]:
ratings.shape

(100003, 3)

In [22]:
import numpy as np 
movie_property = ratings.groupby("movie_id").agg({'ratings':[np.size,np.mean]}) 

In [23]:
movie_property.head()

ratings          
            size      mean
movie_id                  
1          452.0  3.878319
2          131.0  3.206107
3           90.0  3.033333
4          209.0  3.550239
5           86.0  3.302326

In [24]:
movieNumRatings = pd.DataFrame(movie_property['ratings']['size'])

In [25]:
movieNumRatings.head(8)

,size
movie_id,
1,452.0
2,131.0
3,90.0
4,209.0
5,86.0
6,26.0
7,392.0
8,219.0


In [29]:
normalizing_Movie_Ratings = movieNumRatings.apply(lambda x: (x-np.min(x))/(np.max(x)-np.min(x)))
np.max(movieNumRatings['size'])

584.0

In [30]:
### Movie Popularity ( how many people watched it.)
normalizing_Movie_Ratings.head(8)

,size
movie_id,
1,0.773585
2,0.222985
3,0.152659
4,0.356775
5,0.145798
6,0.042882
7,0.670669
8,0.373928


In [32]:
movie_dict = { }

with open("ml-100k/u.item") as f : 
    for line in f : 
        
        fields = line.strip("\n").split("|")
        
        movieID = int(fields[0])
        name = fields[1].strip()
        genres = fields[5:25]
        genres = list(map(int,genres))
        movie_dict[movieID] = (name,genres,normalizing_Movie_Ratings.loc[movieID].get('size'),
                              movie_property.loc[movieID].ratings.get('mean'))


In [33]:
movie_dict[1]

('Toy Story (1995)',
 [0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 0.7735849056603774,
 3.8783185840707963)

In [35]:
movie_dict[2]

('GoldenEye (1995)',
 [0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
 0.22298456260720412,
 3.2061068702290076)

        Now Let's define a function that computes the "distance" between two movies based on how similar their genres are, and how similar their popularity is

In [15]:
from scipy import spatial 
spatial.distance.cosine([0,1,1,0,0],[1,0,1,1,1])

0.6464466094067263

In [16]:
def computeDistance(a,b):
    genresA = a[1]
    genresB = b[1]
    genreDistance = spatial.distance.cosine(genresA,genresB)
    popularityA = a[2]
    popularityB = b[2]
    popularityDistance = abs(popularityA-popularityB)
    return genreDistance + popularityDistance

In [17]:
computeDistance(movie_dict[2],movie_dict[4])

0.8004574042309892

In [18]:
print(movie_dict[2][0])
print(movie_dict[4][0])

GoldenEye (1995)
Get Shorty (1995)


In [19]:
def getNeighbours(movieID,k):
    distance = []
    for movie in movie_dict : 
        if (movie != movieID) : 
            dist = computeDistance(movie_dict[movieID],movie_dict[movie])
            distance.append((movie,dist))
    distance.sort(key=lambda x:x[1])
    neighbours = [ distance[n][0] for n in range(k)]
    return neighbours

In [20]:
k = 10
avgRating = 0
neighbours = getNeighbours(50,k)
print(f"10 nearest Neighbours of {movie_dict[50][0]} : ")
c = 1
for neighbour in neighbours :
    avgRating += movie_dict[neighbour][3]
    print(f"{c}. {movie_dict[neighbour][3]:.4f}",movie_dict[neighbour][0],sep='\t')
    c += 1
avgRating /= k 
print(f"\n\navgRating of all neighbours = {avgRating:.2}")

10 nearest Neighbours of Star Wars (1977) : 
1. 4.0079	Return of the Jedi (1983)
2. 4.2065	Empire Strikes Back, The (1980)
3. 3.4382	Independence Day (ID4) (1996)
4. 3.6603	Star Trek: First Contact (1996)
5. 4.2524	Raiders of the Lost Ark (1981)
6. 3.6570	English Patient, The (1996)
7. 3.2322	Starship Troopers (1997)
8. 4.1728	Princess Bride, The (1987)
9. 3.7203	Jurassic Park (1993)
10. 3.9305	Indiana Jones and the Last Crusade (1989)


avgRating of all neighbours = 3.8


In [21]:
  # 29, 50

In [22]:
import matplotlib.pyplot as plt
%matplotlib inline
